In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import gc
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)

In [2]:
PATH_TO_DATA = ('D:/Py/DataFrames/Home_Credit_Default_Risk(KAGGLE)/')

# Preprocessing

In [3]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=False):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [4]:
# Preprocess application_train.csv and application_test.csv
def application_train_test():
    df = pd.read_csv(os.path.join(PATH_TO_DATA, 'application_train.csv'))
    test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'application_test.csv'))
    df = df.append(test_df).reset_index(drop=True)
    # Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    
    # Preprocessing
    df['NAME_FAMILY_STATUS'].replace('Unknown', 'Single / not married', inplace=True)
    df['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace=True)
    # NaN values for DAYS_EMPLOYED: 365243 -> nan
    for col in ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']:
        df[col].replace(365243, np.nan, inplace=True)
    
    # Categorical features with Binary encode (0 or 1)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    
    # OHE for other categorical features
    df, cat_cols = one_hot_encoder(df, nan_as_category=False)

    
    # [FEATURES]
    df['birth_employed_gap'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
    df['birth_registration_gap'] = df['DAYS_BIRTH'] - df['DAYS_REGISTRATION']
    df['birth_car_age_gap'] = df['DAYS_BIRTH'] - df['OWN_CAR_AGE']
    df['birth_last_phone_gap'] = df['DAYS_BIRTH'] - df['DAYS_LAST_PHONE_CHANGE']
    df['last_phone_employed_perc'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
    
    df['PAYMENT_RATE'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['CREDIT_INCOME_PERC'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['GOODS_PRICE_INCOME_PERC'] = df['AMT_GOODS_PRICE'] / df['AMT_INCOME_TOTAL']
    df['ANNUITY_GOODS_PRICE_PERC'] = df['AMT_ANNUITY'] / df['AMT_GOODS_PRICE']
    df['GOODS_PRICE_CREDIT_PERC'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['INCOME_PER_CHILD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN']) # to avoid +-np.inf
    df['CREDIT_PER_PERSON'] = df['AMT_CREDIT'] / df['CNT_FAM_MEMBERS']
    df['CREDIT_PER_CHILD'] = df['AMT_CREDIT'] / (1 + df['CNT_CHILDREN']) # to avoid +-np.inf
    df['cnt_non_child'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
    df['child_to_non_child_ratio'] = df['CNT_CHILDREN'] / df['cnt_non_child']
    df['income_per_non_child'] = df['AMT_INCOME_TOTAL'] / df['cnt_non_child']
    df['credit_per_non_child'] = df['AMT_CREDIT'] / df['cnt_non_child']
    
    flags = [_f for _f in df.columns if 'FLAG_' in _f]
    for agg in ['sum', 'mean']:
        df['flags_{}'.format(agg)] = eval('np.{}'.format(agg))(df[flags], axis=1)
    
    sources = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
    for agg in ['sum', 'mean', 'median', 'std']:
        df['ext_sources_{}'.format(agg)] = eval('np.{}'.format(agg))(df[sources], axis=1)
    df['ext_sources_prod'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    
    df.drop([
        'FLAG_DOCUMENT_2','FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
        'FLAG_DOCUMENT_6','FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
        'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
        'FLAG_DOCUMENT_12','FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
        'FLAG_DOCUMENT_15','FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
        'FLAG_DOCUMENT_18','FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
        'FLAG_DOCUMENT_21'], axis=1, inplace=True) #useless
    
    
    del test_df
    gc.collect()
    
    return df

In [5]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance():
    bureau = pd.read_csv(os.path.join(PATH_TO_DATA, 'bureau.csv'))
    bb = pd.read_csv(os.path.join(PATH_TO_DATA, 'bureau_balance.csv'))
    
    # [FEATURES]
    bureau['CREDIT_ACTIVE_BINARY'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
    bureau['CREDIT_ENDDATE_BINARY'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)
    
    bb, bb_cat_features = one_hot_encoder(bb)
    bureau, bureau_cat_features = one_hot_encoder(bureau)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_agregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat_features: bb_agregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_agregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, on='SK_ID_BUREAU')
    bureau.drop('SK_ID_BUREAU', axis=1, inplace=True)
    del bb, bb_agg
    gc.collect()
    
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['count', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean', 'sum'],
        'AMT_CREDIT_SUM': ['mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean', 'sum'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean', 'sum'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        
        'CREDIT_ACTIVE_BINARY': ['mean'],
        'CREDIT_ENDDATE_BINARY': ['mean']
    }
    
    cat_aggregations = {}
    for col in bureau_cat_features: cat_aggregations[col] = ['mean']
    for col in bb_cat_features: cat_aggregations[col + '_MEAN'] = ['mean']
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BUREAU_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1] for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1] for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, on='SK_ID_CURR')
    del closed, closed_agg
    
    
    del bureau
    gc.collect()
    
    return bureau_agg

In [6]:
# Preprocess credit_card_balance.csv
def credit_card_balance():
    cc = pd.read_csv(os.path.join(PATH_TO_DATA, 'credit_card_balance.csv'))
    
    
    # Preprocessing
    for col in ['AMT_BALANCE', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT',
                'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE']:
        cc[col] = cc[col].apply(lambda x: x if x > 0 else 0) 
    cc, cc_cat_features = one_hot_encoder(cc)
    
    
    # [FEATURES]
    cc['dpddef_dpd_gap'] = cc['SK_DPD_DEF'] - cc['SK_DPD']
    
    # General aggregations
    cc.drop('SK_ID_PREV', axis=1, inplace=True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1] for e in cc_agg.columns.tolist()])
    
    # [FEATURES]
    cc_agg['cc_count'] = cc.groupby('SK_ID_CURR').size()
    
    
    del cc
    gc.collect()
    
    return cc_agg

In [7]:
# Preprocess installments_payments.csv
def installments_payments():
    ins = pd.read_csv(os.path.join(PATH_TO_DATA, 'installments_payments.csv'))
    ins, ins_cat_features = one_hot_encoder(ins)
    
    
    # [FEATURES]
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_PAYMENT'] - ins['AMT_INSTALMENT']
    ins['PAID_OVER'] = (ins['PAYMENT_DIFF'] > 0).astype(int)
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DPD_LATE'] = (ins['DPD'] > 0).astype(int)
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum', 'std'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum', 'std'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum', 'std'],
        
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'std'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'std'],
        'PAID_OVER': ['mean', 'sum'],
        'DPD': ['max', 'mean', 'sum', 'std'],
        'DPD_LATE': ['mean', 'sum'],
        'DBD': ['max', 'mean', 'sum', 'std'],
    }
    
    for col in ins_cat_features: aggregations[col] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # [FEATURES]
    ins_agg['install_count'] = ins.groupby('SK_ID_CURR').size()
    
    
    del ins
    gc.collect()
    
    return ins_agg

In [8]:
# Preprocess POS_CASH_balance.csv
def pos_cash():
    pos = pd.read_csv(os.path.join(PATH_TO_DATA, 'POS_CASH_balance.csv'))
    pos, pos_cat_features = one_hot_encoder(pos)
    
    
    # Some aggregations
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for col in pos_cat_features: aggregations[col] = ['mean']
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # [FEATURES]
    pos_agg['pos_cash_acc_count'] = pos.groupby('SK_ID_CURR').size()
    
    del pos
    gc.collect()
    
    return pos_agg

In [9]:
# Preprocess previous_applications.csv
def previous_application():
    prev = pd.read_csv(os.path.join(PATH_TO_DATA, 'previous_application.csv'))
    
    
    # Preprocessing
    # Days 365243 values -> nan
    prev.replace(365243, np.nan, inplace= True)
    prev, prev_cat_features = one_hot_encoder(prev)
    
    
    # [FEATURES]
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    prev['PAYMENT_RATE'] = prev['AMT_ANNUITY'] / prev['AMT_CREDIT'] # my
    prev['ANNUITY_GOODS_PRICE_PERC'] = prev['AMT_ANNUITY'] / prev['AMT_GOODS_PRICE'] #my
    
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['max', 'mean'],
        'AMT_APPLICATION': ['max', 'mean'],
        'AMT_CREDIT': ['max', 'mean'],
        'AMT_DOWN_PAYMENT': ['max', 'mean'],
        'AMT_GOODS_PRICE': ['max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['max', 'mean'],
        'RATE_DOWN_PAYMENT': ['max', 'mean'],
        'DAYS_DECISION': ['max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        
        'APP_CREDIT_PERC': ['max', 'mean'],
        'PAYMENT_RATE': ['max', 'mean'],
        'ANNUITY_GOODS_PRICE_PERC': ['max', 'mean']
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for col in prev_cat_features: cat_aggregations[col] = ['mean']
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, on='SK_ID_CURR')
    del approved, approved_agg
    gc.collect()
    
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, on='SK_ID_CURR')
    
    
    del refused, refused_agg, prev
    gc.collect()
    
    return prev_agg

# Merge and Save

In [25]:
%%time
df = application_train_test()

bureau = bureau_and_balance()
df = df.join(bureau, on='SK_ID_CURR')
del bureau
gc.collect()

prev = previous_application()
df = df.join(prev, on='SK_ID_CURR')
del prev
gc.collect()

pos = pos_cash()
df = df.join(pos, on='SK_ID_CURR')
del pos
gc.collect()

ins = installments_payments()
df = df.join(ins, on='SK_ID_CURR')
del ins
gc.collect()

cc = credit_card_balance()
df = df.join(cc, on='SK_ID_CURR')
del cc
gc.collect()

print('Full DF shape: ', df.shape)

Full DF shape:  (356251, 761)
Wall time: 2min 23s


In [26]:
df.to_csv(os.path.join(PATH_TO_DATA, 'df_all_files.csv'), index=False)

In [33]:
df.head(3)

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_3,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,HOUR_APPR_PROCESS_START,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,SK_ID_CURR,TARGET,TOTALAREA_MODE,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_In

---

# Feature Selection

1) Remove missing values

In [10]:
df = pd.read_csv(os.path.join(PATH_TO_DATA, 'df_all_files.csv'))
train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
test.drop('TARGET', axis=1, inplace=True)
del df
gc.collect()

0

In [11]:
train_missing = (train.isnull().sum() / len(train)).sort_values(ascending = False)
train_missing.head()

REFUSED_RATE_DOWN_PAYMENT_MAX     0.853116
REFUSED_AMT_DOWN_PAYMENT_MAX      0.853116
REFUSED_RATE_DOWN_PAYMENT_MEAN    0.853116
REFUSED_AMT_DOWN_PAYMENT_MEAN     0.853116
CC_AMT_PAYMENT_CURRENT_var        0.802870
dtype: float64

In [12]:
test_missing = (test.isnull().sum() / len(test)).sort_values(ascending = False)
test_missing.head()

REFUSED_AMT_DOWN_PAYMENT_MAX      0.847468
REFUSED_RATE_DOWN_PAYMENT_MEAN    0.847468
REFUSED_RATE_DOWN_PAYMENT_MAX     0.847468
REFUSED_AMT_DOWN_PAYMENT_MEAN     0.847468
CC_AMT_PAYMENT_CURRENT_var        0.774680
dtype: float64

In [13]:
train_missing = train_missing.index[train_missing > 0.80]
test_missing = test_missing.index[test_missing > 0.80]

all_missing = list(set(set(train_missing) | set(test_missing)))
print('There are %d columns with more than 80%% missing values' % len(all_missing))

There are 24 columns with more than 80% missing values


In [14]:
train.drop(all_missing, axis=1, inplace=True)
test.drop(all_missing, axis=1, inplace=True)
print(train.shape, test.shape)

(307507, 737) (48744, 736)


2) Remove Collinear Variables

In [15]:
# i am dealing it with such a limited section of the data (~33%) because of waiting
mini_train = train.sample(100000)

In [16]:
%%time
# Absolute value correlation matrix
corr_matrix = mini_train.corr().abs()

Wall time: 6min 9s


In [17]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head(3)

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_3,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,HOUR_APPR_PROCESS_START,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,SK_ID_CURR,TARGET,TOTALAREA_MODE,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_In

In [18]:
threshold = 0.90

to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to remove.' % (len(to_drop)))

There are 183 columns to remove.


In [19]:
to_drop

['AMT_GOODS_PRICE',
 'APARTMENTS_MEDI',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_MODE',
 'COMMONAREA_MEDI',
 'COMMONAREA_MODE',
 'ELEVATORS_MEDI',
 'ELEVATORS_MODE',
 'ENTRANCES_MEDI',
 'ENTRANCES_MODE',
 'FLOORSMAX_MEDI',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MEDI',
 'FLOORSMIN_MODE',
 'LANDAREA_MEDI',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_AVG',
 'LIVINGAREA_MEDI',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MEDI',
 'NONLIVINGAREA_MODE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'REGION_RATING_CLIENT_W_CITY',
 'TOTALAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE',
 'HOUSETYPE_MODE_block of flats',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'NAME_INCOME_TYPE_Pensioner',
 'birth_car_age_gap',
 'birth_last_phone_gap',
 'GOODS_PRICE_INCOME_PERC',
 'INCOME_PER_PERSON',
 'INCOME_PER_CHILD',
 'child_to_non_chi

In [20]:
train.drop(to_drop, axis=1, inplace=True)
test.drop(to_drop, axis=1, inplace=True)
del mini_train
gc.collect()
print(train.shape, test.shape)

(307507, 554) (48744, 553)


In [21]:
train = train.append(test).reset_index(drop=True)
train.to_csv(os.path.join(PATH_TO_DATA, 'df_all_files_with_fs.csv'), index=False)

del train, test, train_missing, test_missing
gc.collect()

14